In [1]:
# Importing necessary packages.
import pandas as pd
import csv
import numpy as np

In [2]:
import dask.dataframe as dd

In [3]:
#from dask.distributed import Client, progress
#client = Client(n_workers=2, threads_per_worker=1, memory_limit='4GB')
#client

### Load NBER Categories

In [4]:
# f_name = "application.tsv"
# f_name = "cpc_group.tsv"
# f_name = "cpc_subgroup.tsv"
# f_name = "cpc_subsection.tsv"
# f_name = "cpc_current.tsv"
# f_name = "mainclass.tsv"
# f_name = "mainclass_current.tsv"
# f_name = "subclass.tsv"
# f_name = "subclass_current.tsv"
f_name = "nber.tsv"
dtypes = {'category_id': np.int8, 'subcategory_id': np.int8}
nber = pd.read_csv(f_name, delimiter="\t", dtype=dtypes, quoting = csv.QUOTE_NONNUMERIC)

In [5]:
f_name = "nber_subcategory.tsv"
nber_cat = pd.read_csv(f_name, delimiter="\t", dtype=dtypes, quoting = csv.QUOTE_NONNUMERIC)

In [6]:
# nber has unique patent ids
nber['patent_id'].nunique()

5105937

In [7]:
nber = nber.merge(nber_cat, left_on = 'subcategory_id', right_on = 'id', how = 'left')

In [8]:
nber.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5105937 entries, 0 to 5105936
Data columns (total 6 columns):
uuid              object
patent_id         object
category_id       int8
subcategory_id    int8
id                float64
title             object
dtypes: float64(1), int8(2), object(3)
memory usage: 204.5+ MB


In [9]:
# drop subcategory id 
nber = nber.drop(columns=['id'])

In [10]:
nber = nber.rename(columns={"title": "subcategory_title"})

In [11]:
nber.dtypes

uuid                 object
patent_id            object
category_id            int8
subcategory_id         int8
subcategory_title    object
dtype: object

In [12]:
nber.sample(5)

,uuid,patent_id,category_id,subcategory_id,subcategory_title
3461129,oeh0mkcofollf2wq40kmjfcki,4171136,2,24,Information Storage
4759597,xkk6nnug03k2chlupb5fu3iha,7790501,4,46,Semiconductor Devices
11821,030hmjcfjjcix8yzst117n0xc,5093778,2,22,Computer Hardware & Software
4816376,xyxvjrpweare0z2cq90e1xtvn,6855126,3,32,Surgery & Med Inst.
2985904,l1vl06c8ilbn4g5xhd9r9nv63,7756437,4,49,Miscellaneous


In [13]:
# nber[nber['patent_id'] == '4433370']

In [14]:
# important step
nber = nber.astype({'patent_id': 'str'})

### load patent files

In [15]:
f_name = "patent.tsv"
dtypes = {'num_claims': np.int16, 'withdrawn': np.float32}
patent = pd.read_csv(f_name, delimiter="\t", dtype=dtypes, parse_dates = ['date'])

/afs/crc.nd.edu/user/y/yniu/.local/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3058: DtypeWarning: Columns (0,2,5) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [16]:
patent.head()

,id,type,number,country,date,abstract,title,kind,num_claims,filename,withdrawn
0,10000000,utility,10000000,US,2018-06-19,A frequency modulated (coherent) laser detecti...,Coherent LADAR using intra-pixel quadrature de...,B2,20,ipg180619.xml,NaN
1,10000001,utility,10000001,US,2018-06-19,The injection molding machine includes a fixed...,Injection molding machine and mold thickness c...,B2,12,ipg180619.xml,NaN
2,10000002,utility,10000002,US,2018-06-19,The present invention relates to: a method for...,Method for manufacturing polymer film and co-e...,B2,9,ipg180619.xml,NaN
3,10000003,utility,10000003,US,2018-06-19,The invention relates to a method for producin...,Method for producing a container from a thermo...,B2,18,ipg180619.xml,NaN
4,10000004,utility,10000004,US,2018-06-19,The present invention relates to provides a do...,"Process of obtaining a double-oriented film, c...",B2,6,ipg180619.xml,NaN


In [17]:
patent.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7430873 entries, 0 to 7430872
Data columns (total 11 columns):
id            object
type          object
number        object
country       object
date          datetime64[ns]
abstract      object
title         object
kind          object
num_claims    int16
filename      object
withdrawn     float32
dtypes: datetime64[ns](1), float32(1), int16(1), object(8)
memory usage: 552.8+ MB


In [18]:
# patent['country'].unique()
patent = patent.astype({'id': 'str'})

In [19]:
# patent = patent[['id', 'type', 'date']]

In [20]:
patent.sample(5)

,id,type,number,country,date,abstract,title,kind,num_claims,filename,withdrawn
2204381,5442484,utility,5442484,US,1995-08-15,A retro-focus-type lens is provided from a la...,Retro-focus type lens and projection-type disp...,A,13,pftaps19950815_wk33.zip,0.0
3038936,6285142,utility,6285142,US,2001-09-04,A display apparatus of multi-mode cathode ray...,Display apparatus having a horizontal screen s...,A,30,pftaps20010904_wk36.zip,0.0
5970734,9230288,utility,9230288,US,2016-01-05,Information can be provided in a convenient ma...,User-specific event popularity map,B2,18,ipg160105.xml,0.0
6268284,9529739,utility,9529739,US,2016-12-27,The present invention relates to a cable (phys...,Communications interface between two non-compl...,B2,16,ipg161227.xml,0.0
6612893,9876713,utility,9876713,US,2018-01-23,Embodiments relate to cross-domain service req...,Cross-domain service request placement in a so...,B2,16,ipg180123.xml,0.0


In [21]:
patent.groupby('type').count()

,id,number,country,date,abstract,title,kind,num_claims,filename,withdrawn
type,,,,,,,,,,
TVPP,3,3,3,3,3,3,3,3,3,3
defensive publication,509,509,509,509,0,509,509,509,509,509
design,645674,645674,645674,645674,22,645674,645674,645674,645674,630205
plant,27887,27887,27887,27887,27858,27887,27887,27887,27887,27311
reissue,19227,19227,19227,19227,18929,19227,19227,19227,19227,18965
statutory invention registration,2258,2258,2258,2258,2251,2258,2258,2258,2258,2257
utility,6735315,6735315,6735315,6735315,6734508,6735315,6735315,6735315,6735315,6580496


In [22]:
# patent[patent['id'] == 4433370]

### select only utility patents

In [23]:
patent = patent[patent['type'] == 'utility']

In [24]:
# patent[patent['id'].str.contains("4433370")]

In [25]:
patent = patent.merge(nber,left_on = "id", right_on = "patent_id", how= "left")

### 75% of patent have NBER categories, which is almost every patent before 2015

In [26]:
patent[~(patent['subcategory_id'].isnull())].shape[0]/ patent.shape[0]

0.7555037886127078

In [27]:
patent['year'] = patent['date'].dt.year

In [28]:
patent[patent['subcategory_id'].isnull()].sample(5)

,id,type,number,country,date,abstract,title,kind,num_claims,filename,withdrawn,uuid,patent_id,category_id,subcategory_id,subcategory_title,year
6551321,9814828,utility,9814828,US,2017-11-14,An intravenous (IV) fluid monitoring system an...,Method and apparatus for preparing and monitor...,B2,20,ipg171114.xml,0.0,NaN,NaN,NaN,NaN,NaN,2017
638362,10643297,utility,10643297,US,2020-05-05,One embodiment provides for a graphics process...,Dynamic precision management for integer deep ...,B2,20,ipg200505.xml,0.0,NaN,NaN,NaN,NaN,NaN,2020
6219219,9480445,utility,9480445,US,2016-11-01,"An elevating column for an X-ray apparatus, th...",Elevating column and method of controlling ele...,B2,26,ipg161101.xml,0.0,NaN,NaN,NaN,NaN,NaN,2016
6667684,9931963,utility,9931963,US,2018-04-03,An electric motor control system for a vehicle...,Vehicle speed control systems and methods,B2,20,ipg180403.xml,0.0,NaN,NaN,NaN,NaN,NaN,2018
561468,10565777,utility,10565777,US,2020-02-18,A method for reducing discomfort when viewing ...,Field of view (FOV) throttling of virtual real...,B2,20,ipg200218.xml,0.0,NaN,NaN,NaN,NaN,NaN,2020


In [29]:
# we have substantial NBER category missing, starting from May 2015
patent[patent['subcategory_id'].isnull()].groupby('year')['id'].count()

year
1976         1
1986         1
2002         4
2003         1
2004         3
2005         7
2006         4
2007         7
2008         1
2009        12
2010        32
2011        45
2012        82
2013       105
2014       176
2015    181393
2016    304126
2017    320003
2018    308853
2019    355923
2020    175980
Name: id, dtype: int64

In [30]:
patent.columns

Index(['id', 'type', 'number', 'country', 'date', 'abstract', 'title', 'kind',
       'num_claims', 'filename', 'withdrawn', 'uuid', 'patent_id',
       'category_id', 'subcategory_id', 'subcategory_title', 'year'],
      dtype='object')

### load citations 

In [31]:
f_name = "uspatentcitation.tsv"
# # dtypes = {'sequence': np.int16}
# # citation = pd.read_csv(f_name, delimiter="\t", dtype=dtypes)
citation = pd.read_csv(f_name, delimiter="\t") # , usecols = ['patent_id','citation_id','date'], dtype=dtypes

In [32]:
# use sample for experiments // or using the file below for reading partial data
# citation = citation.sample(frac=0.1).compute()

In [33]:
# read partial data for speed/ prototypring 

# f_name = "uspatentcitation.tsv"

# Take every N-th (in this case 10th) row
# n = 10

# Count the lines or use an upper bound
# num_lines = sum(1 for l in open(f_name))

# The row indices to skip - make sure 0 is not included to keep the header!
# skip_idx = [x for x in range(1, num_lines) if x % n != 0]

# Read the data
# citation = pd.read_csv(f_name, skiprows=skip_idx, delimiter="\t", usecols = ['patent_id','citation_id','date'], dtype=dtypes)

In [34]:
citation = citation[['patent_id','citation_id']]
# 'date' actually date can be added later 

In [35]:
citation = citation.astype({'patent_id': 'str', 'citation_id': 'str'})

In [36]:
citation.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 111026439 entries, 0 to 111026438
Data columns (total 2 columns):
patent_id      object
citation_id    object
dtypes: object(2)
memory usage: 1.7+ GB


In [37]:
citation.dtypes

patent_id      object
citation_id    object
dtype: object

In [38]:
citation.head()

,patent_id,citation_id
0,5354551,4875247
1,D674253,D519335
2,D809697,1963218
3,D808172,7207426
4,8683318,6642945


In [39]:
# citation['patent_year'] = citation['date'].astype(str).str[:4]

In [40]:
# citation = citation.dropna(subset=['patent_year'])

In [41]:
# citation = citation.astype({'year': 'int16'})

In [42]:
# citation = citation[['patent_id','citation_id','year']]

In [43]:
citation.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 111026439 entries, 0 to 111026438
Data columns (total 2 columns):
patent_id      object
citation_id    object
dtypes: object(2)
memory usage: 1.7+ GB


In [44]:
# # f_name = "uspc.tsv"
# # f_name = "ipcr.tsv"
# # f_name = "patent.tsv"
# # f_name = "usapplicationcitation.tsv"
# f_name = "uspatentcitation.tsv"
# chunksize = 10 ** 6
# count = 1
# n_obs = 0
# for df in pd.read_csv(f_name, delimiter="\t", chunksize=chunksize, quoting=csv.QUOTE_NONNUMERIC):
#     print('processing chunk: ' + str(count))
#     n_obs += len(df)
#     count += 1
# # Print summary of data: number of observations, columns, and each variable data type
# print(n_obs)
# print(df.dtypes)

In [45]:
# Print summary of data: number of columns, observations, and each variable data type
# print(len(df))
# df.info()
# Print basic summary statistics for numerical variables
# print(df.describe(exclude=[np.number]))

In [46]:
# df.to_csv('usapplicationcitation_sample.csv', index=False)  
# df.to_csv('patent_sample.csv', index=False)
# df.to_csv('uspatentcitation_sample.csv', index=False)
# pd.read_csv('usapplicationcitation_sample.csv')
# pd.read_csv('patent_sample.csv')
# pd.read_csv('uspatentcitation_sample.csv')

In [47]:
# Print first five observations
# df.head(20)
# df.date.max()

## merge datasets

In [48]:
citation.dtypes

patent_id      object
citation_id    object
dtype: object

In [49]:
nber.dtypes

uuid                 object
patent_id            object
category_id            int8
subcategory_id         int8
subcategory_title    object
dtype: object

In [50]:
patent.dtypes

id                           object
type                         object
number                       object
country                      object
date                 datetime64[ns]
abstract                     object
title                        object
kind                         object
num_claims                    int16
filename                     object
withdrawn                   float32
uuid                         object
patent_id                    object
category_id                 float64
subcategory_id              float64
subcategory_title            object
year                          int64
dtype: object

### merge with patents and keep only utility patents

In [51]:
citation = citation.merge(patent[['id','year','subcategory_id']], left_on = "patent_id", right_on = "id", how = 'left')

In [52]:
citation = citation.rename(columns={"year": "citing_year",'subcategory_id':'citing_subcategory_id'})

In [53]:
citation.head(5)

,patent_id,citation_id,id,citing_year,citing_subcategory_id
0,5354551,4875247,5354551,1994.0,31.0
1,D674253,D519335,NaN,NaN,NaN
2,D809697,1963218,NaN,NaN,NaN
3,D808172,7207426,NaN,NaN,NaN
4,8683318,6642945,8683318,2014.0,22.0


In [54]:
# citation['']citation['date'].astype(str).str[:4]

# 10% of citations do not have a type..... ok

In [55]:
citation[citation['id'].isnull()].shape[0] / citation.shape[0]

0.10090528076830421

In [56]:
citation = citation.merge(patent[['id','year','subcategory_id']], left_on = "citation_id", right_on = "id", how = 'left')
citation = citation.rename(columns={"year": "cited_year",'subcategory_id':'cited_subcategory_id'})

In [57]:
# 20% of patents don't have a subcategory_id 
citation[citation['cited_subcategory_id'].isnull()].shape[0] / citation.shape[0]

0.2086653432161325

In [58]:
citation.dtypes

patent_id                 object
citation_id               object
id_x                      object
citing_year              float64
citing_subcategory_id    float64
id_y                      object
cited_year               float64
cited_subcategory_id     float64
dtype: object

In [59]:
citation.head()

,patent_id,citation_id,id_x,citing_year,citing_subcategory_id,id_y,cited_year,cited_subcategory_id
0,5354551,4875247,5354551,1994.0,31.0,4875247,1989.0,69.0
1,D674253,D519335,NaN,NaN,NaN,NaN,NaN,NaN
2,D809697,1963218,NaN,NaN,NaN,NaN,NaN,NaN
3,D808172,7207426,NaN,NaN,NaN,7207426,2007.0,69.0
4,8683318,6642945,8683318,2014.0,22.0,6642945,2003.0,22.0


### select before 2015, since the category stopped at may 2015

In [60]:
citation[citation['citing_year'] < 2015].shape[0] / citation.shape[0]

0.590824163963324

In [61]:
citation = citation[citation['citing_year'] < 2015]

In [62]:
citation['cite_year_gap'] = citation['citing_year'] - citation['cited_year']

In [63]:
# citation[['cite_year_gap']].hist()

# limit forward citation towards less than 10 years 

In [64]:
citation = citation[citation['cite_year_gap']<=10]

### calculate directional citation flows

In [65]:
list_subcat = list(citation['citing_subcategory_id'].dropna().unique())

In [66]:
list_subcat.sort()

In [67]:
df_citation = pd.DataFrame()

for patent_subcategory_id in list_subcat:
    temp = citation[citation['citing_subcategory_id'] == patent_subcategory_id].groupby("cited_subcategory_id").count()\
        [['citing_subcategory_id']].rename(columns={"citing_subcategory_id": patent_subcategory_id})
    df_citation = df_citation.merge(temp, left_index=True, right_index=True, how = "outer")

In [68]:
df_citation = df_citation.replace(np.nan,0)

In [69]:
df_citation

,11.0,12.0,13.0,14.0,15.0,19.0,21.0,22.0,23.0,24.0,...,61.0,62.0,63.0,64.0,65.0,66.0,67.0,68.0,69.0,70.0
cited_subcategory_id,,,,,,,,,,,,,,,,,,,,,
11.0,34450.0,1820.0,67.0,5264,1761.0,7671.0,75,343,162,31,...,1526,18,2148.0,218.0,139.0,47.0,12.0,144.0,2759,4.0
12.0,1217.0,118776.0,1222.0,2779,20559.0,41959.0,1810,2728,6407,1578,...,1574,372,774.0,6087.0,1434.0,1253.0,420.0,991.0,34622,2.0
13.0,75.0,1017.0,69811.0,544,488.0,18330.0,297,564,164,274,...,354,43,104.0,994.0,219.0,1472.0,319.0,361.0,6639,0.0
14.0,5208.0,4352.0,710.0,145211,25333.0,38513.0,452,918,721,226,...,2347,68,48.0,616.0,52.0,71.0,33.0,84.0,4607,11.0
15.0,1936.0,24014.0,1073.0,20076,316181.0,57656.0,2351,1074,2768,434,...,1749,4557,816.0,5203.0,592.0,277.0,730.0,1080.0,55051,3.0
19.0,7253.0,45981.0,24846.0,31645,49772.0,1135234.0,13429,19804,13330,6111,...,10420,1724,4579.0,13777.0,3213.0,8811.0,3370.0,13823.0,97544,2.0
21.0,62.0,1684.0,258.0,223,1349.0,10307.0,2225094,420243,65889,51423,...,1888,6926,1166.0,5367.0,4074.0,569.0,490.0,1501.0,20288,54.0
22.0,221.0,1974.0,306.0,376,666.0,11344.0,372272,1734424,127017,236960,...,1756,13815,1330.0,3936.0,1437.0,965.0,162.0,1304.0,21528,137.0
23.0,194.0,6054.0,138.0,257,1791.0,10680.0,63174,139370,551711,43767,...,193,10012,193.0,199.0,392.0,163.0,61.0,476.0,12031,103.0


In [70]:
df_citation.to_csv('citation_cumsum.csv', index=True)

In [71]:
#citation[(citation['citing_year']>=start) & (citation['citing_year']<start+5+i*5)]

NameError: name 'start' is not defined

In [ ]:
#citation['citing_year'].unique().max()

In [72]:
for i in range(7):
    start = 1980
    gap = 5
    
    citation_snapshot = pd.DataFrame()

    for patent_subcategory_id in list_subcat:
        temp = citation[(citation['citing_year']>=start) & (citation['citing_year']<start+5+i*5)&
                        (citation['citing_subcategory_id'] == patent_subcategory_id)]\
                        .groupby("cited_subcategory_id").count()\
                        [['citing_subcategory_id']].rename(columns={"citing_subcategory_id": patent_subcategory_id})
        citation_snapshot = citation_snapshot.merge(temp, left_index=True, right_index=True, how = "outer")
        citation_snapshot.to_csv('5_year_citation_snapshot'+ str(start+i*5)+'.csv', index=True)